In [3]:
import pandas as pd
from scipy.spatial import distance
import math
import sys
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
import miniball

In [4]:
freq = pd.read_csv("../utils/word_frequency/bert_word_frequencies.csv")

In [9]:
WORD_INDEX = 1537
def create_miniballs(input_filename, output_filename):
    
    data = pd.read_csv(input_filename, header=None)
    all_word_embeddings = data[list(range(0, 768)) + [1536, 1537]]
    
    #drop duplicate word embeddings
    all_word_embeddings = all_word_embeddings.drop_duplicates(keep='first', subset=list(range(0, 768))).sample(frac=1)
    
    #creating convex hulls or miniballs
    all_word_embeddings = all_word_embeddings.groupby(WORD_INDEX)
    result = pd.DataFrame()
    for name, group in all_word_embeddings:
        n = 768

        current = np.delete(group.values, n, 1)
        current = np.delete(current, n, 1)
        #just take a sample of ten
        current = current[:10]
        C, r2 = miniball.get_bounding_ball(np.array(current, dtype='float'))
      
        result = pd.concat([result, pd.DataFrame([[name, r2]])])
        print(name, r2)

    result.columns=['keyword', 'r2']

    #Join with word frequency data
    result = result.set_index('keyword').join(freq.set_index('index'), how='outer')
    result = result.dropna()
    result.to_csv(output_filename)


In [10]:
create_miniballs("wic_embeddings_wikipedia_corpus.csv", "wic_bounding_balls.csv")

In [11]:
create_miniballs("30k_random_words_embeddings_final.csv", "30k_random_words_bounding_balls.csv")